In [1]:
import torch as tc
import pandas as pd
import numpy as np
from src.bbox import BBox
import open3d as otd
from tqdm import tqdm
import matplotlib.pyplot as plt
otd_vector3d = otd.utility.Vector3dVector
from src.star.star import STAR
from src.curve_utils import CurveUtils
from src.curve_generator import CurveGenerator
from src.mesh_manipulation import save_obj
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")
genders = ['female', 'male']
male = ['male']
female = ['female']

In [2]:
curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
}

In [3]:
subdivided_bodies = tc.load('data/ss20_subdivided_bodies.pt')
gender_curves = tc.load("data/gender_curves.zip")
gender_results = tc.load("data/best_gender_curves.zip")

In [4]:
column_curves_measures = {}

for idx, gender in enumerate(genders):

    template = subdivided_bodies['faces'][gender].to(device)

    column_curves_measures[gender] = []
    best_neck = gender_curves[gender][curve_index['neck_girth']][gender_results[gender]['best']['neck_girth']]
    best_neck = tc.FloatTensor(best_neck).to(device)

    best_waist = gender_curves[gender][curve_index['waist_girth']][gender_results[gender]['best']['waist_girth']]
    best_waist = tc.FloatTensor(best_waist).to(device)
    
    bodies_interator = tqdm(subdivided_bodies['vertices'][gender])
    for body in bodies_interator:

        best_neck_pos = CurveUtils.generate_positions(best_neck, body.to(device))
        best_waist_pos = CurveUtils.generate_positions(best_waist, body.to(device))

        neck_ht = best_neck_pos[:,1].max()
        waist_ht = best_waist_pos[:,1].max()

        neck_width = abs(best_neck_pos[:,2]).max()
        waist_width = abs(best_waist_pos[:,2]).max()

        
        width = (neck_width + (waist_width/2))*1.2
        height = neck_ht - waist_ht
        center = (0, waist_ht + (height/2), -width/2)
        column_box = BBox(length=width, width=width, height=height, center=tc.FloatTensor(center).to(device))
        column_curves = CurveGenerator.calculate_curve(body.to(device), column_box, 0, template, device)      


        all_col_curves = []
        for curve in column_curves[0]:
            positions = CurveUtils.generate_positions(tc.FloatTensor(curve).to(device), body.to(device))
            all_col_curves.append(positions[:,0].abs().mean())
        idmin = tc.FloatTensor(all_col_curves).abs().argmin()
        measure = CurveUtils.linear_distance(tc.FloatTensor(column_curves[2][idmin]))
        column_curves_measures[gender].append(measure)

100%|██████████| 20/20 [00:23<00:00,  1.17s/it]
